In [30]:
import os, json
import ipywidgets
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
res_folder = os.path.abspath('./active_ML/results/')

In [31]:
res_list = []
for fname in os.listdir(res_folder):
    if not fname.endswith('.json'):
        continue
    fpath = os.path.join(res_folder, fname)
    with open(fpath, 'r') as fd:
        res = json.load(fd)
        res.update(y_scaler=fname.replace('morgan_circular', 'morgancircular').split('_')[3])
        if res['model_name'] == 'active':
            res['n_train_mean'] = int(fname.split('_')[2])
        res_list.append(res)
        
data = pd.DataFrame(res_list)
display(data.head())

,data_name,mae_mean,mae_std,model_name,mse_mean,mse_std,n_eval_mean,n_eval_std,n_train_mean,n_train_std,pearsonr_mean,pearsonr_std,r2_score_mean,r2_score_std,std_se_mean,std_se_std,y_scaler
0,1259374,0.109838,0.028559,morgan_circular,0.045888,0.025034,288381.0,0.0,100.0,0.0,0.007314,0.009935,-21.861141,5.129763,2.392497,1.318758,minmax
1,1259374,1.139484,0.279163,active,5.125391,3.572996,57696.0,0.0,20.0,0.0,0.001987,0.009196,-29.822259,13.324770,271.819884,337.643555,norm
2,1259374,8.843995,0.155081,active,288.384283,37.309713,57696.0,0.0,5000.0,0.0,0.005051,0.009871,-31.331421,12.686031,13722.104752,6534.827282,none
3,651958,1.134260,0.395103,active,5.782221,2.694734,33874.0,0.0,500.0,0.0,0.008557,0.016229,-13.824472,9.847121,60.836475,29.788136,norm
4,1259318,0.176657,0.077101,morgan_circular,0.293694,0.211586,171033.0,0.0,20.0,0.0,0.001490,0.004419,-78.672770,43.118508,43.827402,31.733614,minmax


In [34]:
def dropdown_widget(name, opts):
    return ipywidgets.Dropdown(
                        options=opts,
                        value=opts[0],
                        description=name,
                        disabled=False)

def plot_curves(resdata, metric='mae', **filters):
    table = resdata.copy()
    for key, value in filters.items():
        table = table[table[key]==value]
        
    out = ipywidgets.Output()
    with out:
        plt.figure()
        sns.lineplot(x="n_train_mean", y=metric+"_mean", hue="model_name", data=table)
    return out 

def get_plot_widgets(data):
    config_options = ['data_name', 'y_scaler']
    widgets = dict()
    widgets['metric'] = dropdown_widget(name='Metric', opts=['mae', 'pearsonr', 'mse'])
    for key in config_options:
        widgets[key] = dropdown_widget(name=key.replace('_', ' ').capitalize(), opts=list(set(data[key].tolist())))
    return widgets

widgets = get_plot_widgets(data)
drawing_func = lambda **x: plot_curves(data, **x)
ipywidgets.interact(drawing_func, **widgets)

interactive(children=(Dropdown(description='Metric', options=('mae', 'pearsonr', 'mse'), value='mae'), Dropdow…

<function __main__.<lambda>(**x)>